In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import datetime 
import warnings
import csv
warnings.simplefilter(action='ignore', category=FutureWarning)
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cat = pd.read_csv ('../input/competitive-data-science-predict-future-sales/item_categories.csv')
trainset1 = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
testset= pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
trainset1.info()

In [ ]:
items.info()

In [ ]:
item_cat.info()

In [ ]:
shops.info()

In [ ]:
trainset1.shape

In [ ]:
display(trainset1, item_cat, items, shops)

In [ ]:
trainset2=pd.merge(items, trainset1)

In [ ]:
trainset3=pd.merge(item_cat, trainset2)

In [ ]:
train=pd.merge(shops, trainset3)

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train['shop_name'].unique()

In [ ]:
train['shop_id'].unique()

In [ ]:
train['item_category_name'].unique()

In [ ]:
train['item_name'].unique()

In [ ]:
train['item_id'].unique()

In [ ]:
train['shop_name'].nunique()

In [ ]:
train['shop_id'].nunique()

In [ ]:
train['item_category_name'].nunique()

In [ ]:
train['item_name'].nunique()

In [ ]:
train['item_id'].nunique()

In [ ]:
train_set=train.copy()

In [ ]:
corrmat=train_set.corr()
corrmat

In [ ]:
plt.figure(figsize=(15, 15))
sns.set(font_scale=2)
sns.heatmap(train_set.corr(), annot=True, cmap="Blues", fmt= ".2g")

In [ ]:
negative_price=train_set[train_set["item_price"] < 0]
negative_price

In [ ]:
price_2973=train_set[(train_set["shop_id"]== 32) & (train_set["item_category_name"]=="Игры - PS3") & (train_set["item_id"]== 2973 ) & (train_set["date_block_num"]== 4)]
price_2973

In [ ]:
price_mean=price_2973[(price_2973.shop_id==32)&(price_2973.item_id==2973)&(price_2973.date_block_num==4)&(price_2973.item_price>0)]
price_mean.item_price.mean()

In [ ]:
train_set.at[1644072,'item_price']= 1849

In [ ]:
negative_item_cnt_day=train_set[train_set["item_cnt_day"] < 0]
negative_item_cnt_day

In [ ]:
cond = train_set['item_cnt_day'] < 0
check= train_set.loc[cond,'item_cnt_day'] = 0
check

In [ ]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_price'])
plt.title('Outliers in item_price', fontsize = 40)

In [ ]:
print("Maximum possible price",train_set['item_price'].mean() + 3*train_set['item_price'].std())
print("Lowest possible price",train_set['item_price'].mean() - 3*train_set['item_price'].std())

In [ ]:
hp=max(train_set.item_price)
print("Highest price :", hp)
lp=min(train_set.item_price)
print("Lowest price: ", lp)

In [ ]:
highest_price=train_set[train_set["item_price"] > 6080]
highest_price

In [ ]:
train_set = train_set[train_set['item_price'] < 100000]

In [ ]:
plt.figure(figsize=(10, 15))
train_set.boxplot(['item_cnt_day'])
plt.title('Outliers in item_cnt_day', fontsize = 40)

In [ ]:
print("Maximum possible number of products sold",train_set['item_cnt_day'].mean() + 3*train_set['item_cnt_day'].std())
print("Lowest possible number of products sold",train_set['item_cnt_day'].mean() - 3*train_set['item_cnt_day'].std())

In [ ]:
hps=max(train_set.item_cnt_day)
print("Highest number of products sold :", hps)
lps=min(train_set.item_cnt_day)
print("Lowest number of products sold: ", lps)

In [ ]:
highest_price=train_set[train_set["item_cnt_day"] > 9]
highest_price

In [ ]:
train_set = train_set[train_set['item_cnt_day'] < 1000]

In [ ]:
train_set['Sales'] = (train_set["item_price"] * train_set["item_cnt_day"])

In [ ]:
train_set['date'] = pd.to_datetime(train_set['date'])
days = []
months = []
years = []

for day in train_set['date']:
    days.append(day.day)
for month in train_set['date']:
    months.append(month.month)    
for year in train_set['date']:
    years.append(year.year)

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(years)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

In [ ]:
plt.figure(figsize=(40, 50))
x= train_set['item_category_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Product', fontsize = 60)
plt.xlabel('Item categories', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

In [ ]:
highest_selling_itemc= train_set.loc[train_set['item_category_id'] == 12]
highest_selling_itemc

In [ ]:
plt.figure(figsize=(40, 50))
x= train_set['shop_id']
y= train_set['Sales']
sns.barplot(x, y)
plt.title('Highest Selling Shop', fontsize = 60)
plt.xlabel('Shop_id', fontsize = 30)
plt.ylabel('Sales', fontsize = 30)
plt.show()

In [ ]:
highest_selling_shop= train_set.loc[train_set['shop_id'] == 9]
highest_selling_shop

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(months)
plt.title('Sales per Year', fontsize = 40)
plt.xlabel('Year', fontsize = 25)
plt.ylabel('Sales', fontsize = 25)
plt.show()

In [ ]:
sales_by_month = train_set.groupby(['date_block_num'])[["Sales"]].sum()
plt.figure(figsize=(30,30))
sales_by_month.plot()
plt.title('Month with highest Sales in the dataset', fontsize = 10)
plt.xlabel('Months', fontsize = 10)
plt.ylabel('Sales', fontsize = 10)

In [ ]:
sales_max = train_set.groupby(['date_block_num'])['Sales'].sum()
sales_max.max()

In [ ]:
print(sales_max==232615420.90999845)

In [ ]:
highest_sales_month = train_set[train_set["date_block_num"]==23]
highest_sales_month

In [ ]:
highest_store_sales = train_set.groupby(['date_block_num', 'shop_name'])['Sales'].sum()

highest_store_sales.max()

In [ ]:
highest_store_sales=pd.DataFrame(highest_store_sales)
highest_store_sales=highest_store_sales.loc[highest_store_sales['Sales'] == 15730394.000000501]
highest_store_sales

In [ ]:
highest_item_sales = train_set.groupby(['date_block_num', 'item_category_name'])['Sales'].sum()

highest_item_sales.max()

In [ ]:
highest_item_sales=pd.DataFrame(highest_item_sales)
highest_item_sales=highest_item_sales.loc[highest_item_sales['Sales'] == 46487721.9999976]
highest_item_sales

In [ ]:
train_set = train_set[["date_block_num", "shop_id", "item_id", "item_price", "item_cnt_day", "shop_name", "Sales", "item_name"]].groupby(
    ["date_block_num", "shop_id", "item_id"]).agg(
    {"item_cnt_day": "sum", "Sales" : "sum"}).reset_index()
train_set.rename(columns={"item_cnt_day": "item_cnt_month"}, inplace=True)

In [ ]:
test1=pd.merge(items, testset)
test2=pd.merge(item_cat, test1)
test = pd.merge(shops,test2)
test

In [ ]:
test=test.drop("ID",1)

In [ ]:
test.shape

In [ ]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

In [ ]:
train_set = train_set[train_set['shop_id'].isin(test['shop_id'].unique())]
train_set = train_set[train_set['item_id'].isin(test['item_id'].unique())]

In [ ]:
train_set_u = train_set['item_id'].nunique()
test_u = test['item_id'].nunique()
train_set_s = train_set['shop_id'].nunique()
test_s = test['shop_id'].nunique()
print("Total unique item_ids in train_set dataset: ", train_set_u)
print("Total unique item_ids in test dataset: ", test_u )

print("Total unique shop_ids train_set dataset: ", train_set_s)
print("Total unique shop_ids in test dataset: ", test_s)

In [ ]:
test.shape

In [ ]:
test["item_cnt_month"]=" "

In [ ]:
test.info()
train_set.info()

In [ ]:
train_set=train_set.drop("Sales", 1)
train_set=train_set.drop("date_block_num", 1)
test= test.drop("item_category_id", 1)
test= test.drop("item_category_name", 1)
test= test.drop("item_name", 1)
test= test.drop("shop_name", 1)

In [ ]:
train_x = train_set.drop(['item_cnt_month'], axis=1)
train_y = train_set['item_cnt_month']
test_x = test.drop(['item_cnt_month'], axis=1)
model_lgb = LGBMRegressor(colsample_bytree=0.9, learning_rate=0.03, max_depth=6,
              min_child_weight=1, min_split_gain=0.0222415, n_estimators=100,
              num_leaves=32, reg_alpha=0.04, reg_lambda=0.073,
              subsample=0.9)
model_lgb.fit(train_x, train_y)

In [ ]:
sales_prediction = model_lgb.predict(test_x)
sales_prediction

In [ ]:
sales_predict_submission = pd.DataFrame({'ID':test_x.index,'item_cnt_month':sales_prediction})

sales_predict_submission

In [ ]:
 sales_predict_submission.to_csv('./submission.csv', index= False, header= 1)